# Libraries & Constants

In [1]:
__author__ = "Aiello Davide"
import logging
from collections import namedtuple
import random
from typing import Callable
from copy import deepcopy
from itertools import accumulate
from operator import xor         

# Nim Class

In [2]:
Nimply = namedtuple("Nimply", "row, num_objects")

In [3]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    @property
    def k(self) -> int:
        return self._k

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects

# Nim-sum

In [4]:
# algorithm taken from professor's code
def nim_sum(state: Nim) -> int:
    *_, result = accumulate(state.rows, xor)
    return result


def cook_status(state: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = [
        (r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1) if state.k is None or o <= state.k
    ]
    cooked["active_rows_number"] = sum(o > 0 for o in state.rows)       # numero di righe maggiori di zero     
    cooked["shortest_row"] = min((x for x in enumerate(state.rows) if x[1] > 0), key=lambda y: y[1])[0]
    cooked["longest_row"] = max((x for x in enumerate(state.rows)), key=lambda y: y[1])[0]
    cooked["nim_sum"] = nim_sum(state)

    brute_force = list()
    for m in cooked["possible_moves"]:
        tmp = deepcopy(state)
        tmp.nimming(m)
        brute_force.append((m, nim_sum(tmp)))
    cooked["brute_force"] = brute_force

    return cooked

def optimal_strategy(state: Nim) -> Nimply:
    data = cook_status(state)
    return next((bf for bf in data["brute_force"] if bf[1] == 0), random.choice(data["brute_force"]))[0]

# Task 3.3: minmax

In [5]:
def minmax(state: Nim, depth: int, max_turn: bool):
    if sum(o > 0 for o in state.rows) == 0:     
        return -1 if max_turn else 1 
 
    if depth == 0:
        return 0
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1) if state.k is None or o <= state.k]
        
    if max_turn:
        scores = []
        for ply in possible_moves:
            tmp = deepcopy(state)
            tmp.nimming(ply)
            scores.append(minmax(tmp, depth - 1, max_turn=False))
        return max(scores)
    else:
        scores = []
        for ply in possible_moves:
            tmp = deepcopy(state)
            tmp.nimming(ply)
            scores.append(minmax(tmp, depth - 1, max_turn=True))
        return min(scores)

# Main

In [6]:
k = 100
NIM_SIZE = 3

logging.getLogger().setLevel(logging.DEBUG)

def pure_random(state: Nim) -> Nimply: 
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    if(state.rows[row] > state.k):
        num_objects = random.randint(1, state.k)
    else:
        num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)

def minmax_strategy(state, depth=20, max_turn=False):
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1) if state.k is None or o <= state.k]
    scores = []
    for ply in possible_moves:
        tmp = deepcopy(state)
        tmp.nimming(ply)
        scores.append(minmax(tmp, depth, max_turn=max_turn))
        if scores[-1] > 0:
            return ply
    else:                                 
        return pure_random(state)
              
def shortest_row(state: Nim) -> Nimply: # take the shortest row and select random elements if the matches > k otherwise close the row
    row = min((x for x in enumerate(state.rows) if x[1] > 0), key=lambda y: y[1])[0]
    if(state.rows[row] > state.k):
       num_objects = random.randint(1, state.k)
    else:
       num_objects = state.rows[row]
    return Nimply(row, num_objects)


def evaluate(pure_random: Callable, my_strategy: Callable, NUM_MATCHES : int) -> list:
    w = [0, 0]
    turn = [0, 1]
    for t in turn:
        if t == 0:
            opponent = (my_strategy, optimal_strategy)
        else:
            opponent = (optimal_strategy, my_strategy)
        for m in range(NUM_MATCHES):
            nim = Nim(NIM_SIZE, k)
            player = 0
            while nim:
                ply = opponent[player](nim)
                nim.nimming(ply)
                player = 1 - player
            winner = 1 - player
            if winner == t:
                w[t] += 1
    w[0] = w[0] / NUM_MATCHES
    w[1] = w[1] / NUM_MATCHES
    return w

for n_m in [10, 50, 100]:
    w = evaluate(optimal_strategy, minmax_strategy, n_m)
    logging.info(f"NUM_MATCHES : {n_m} - ratio as player1: {w[0]*100}, ratio as player2: {w[1]*100}")


INFO:root:NUM_MATCHES : 10 - ratio as player1: 100.0, ratio as player2: 0.0
INFO:root:NUM_MATCHES : 50 - ratio as player1: 100.0, ratio as player2: 0.0
INFO:root:NUM_MATCHES : 100 - ratio as player1: 100.0, ratio as player2: 0.0
